In [1]:
# load tqdm
# !pip3 install --force https://github.com/chengs/tqdm/archive/colab.zip

In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
df = pd.read_csv(
    '/content/smile-annotations-final.csv',
    names=['id','text','category'])

In [4]:
df.set_index('id', inplace=True)

In [5]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [6]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [7]:
df = df[~df.category.str.contains('\|')]

In [8]:
df = df[df.category != 'nocode']

In [9]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [10]:
label_dict = {l:i for i,l in enumerate(df.category.unique())}
num2label = {v:k for k, v in label_dict.items()}

In [11]:
num2label

{0: 'happy',
 1: 'not-relevant',
 2: 'angry',
 3: 'disgust',
 4: 'sad',
 5: 'surprise'}

In [12]:
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [13]:
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


In [14]:
from sklearn.model_selection import train_test_split

In [15]:
SEED = 42069

In [16]:
Xt, Xv, yt, yv = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=SEED,
    stratify=df.label.values
)

In [17]:
df['type'] = ['tmp'] * df.shape[0]

In [18]:
df.head()

,text,category,label,type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,tmp
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,tmp
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,tmp
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,tmp
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,tmp


In [19]:
df.loc[Xt, 'type'] = 'train'
df.loc[Xv, 'type'] = 'val'

In [20]:
df.groupby(['category', 'label', 'type']).count()

text
category     label type       
angry        2     train    48
                   val       9
disgust      3     train     5
                   val       1
happy        0     train   966
                   val     171
not-relevant 1     train   182
                   val      32
sad          4     train    27
                   val       5
surprise     5     train    30
                   val       5

In [21]:
!pip install transformers >> /dev/null

In [22]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [23]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [24]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [25]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.type=='train'].label.values)

In [26]:
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.type=='val'].label.values)

In [27]:
train_ds = TensorDataset(input_ids_train, attention_masks_train, labels_train)
val_ds = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [28]:
print(len(train_ds), len(val_ds))

1258 223


In [29]:
from transformers import BertForSequenceClassification

In [30]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [31]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [32]:
BATCH_SIZE = 32
EPOCHS = 10

In [33]:
train_dl = DataLoader(
    train_ds,
    sampler=RandomSampler(train_ds),
    batch_size=BATCH_SIZE
)

val_dl = DataLoader(
    val_ds,
    sampler=RandomSampler(val_ds),
    batch_size=BATCH_SIZE
)

In [34]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [35]:
optim = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)

In [36]:
scheduler = get_linear_schedule_with_warmup(
    optim,
    num_warmup_steps=0,
    num_training_steps=len(train_dl)*EPOCHS
)

 Defining Performance Metrics

In [37]:
import numpy as np
from sklearn.metrics import f1_score

In [38]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [39]:
def print_acc_per_class(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    for label in np.unique(labels_flat):
        total_true = preds_flat[labels_flat == label]
        true_positives = total_true[total_true == label]
        print(f'Class: {num2label[label]}')
        print(f'Accuracy: {len(true_positives)}/{len(total_true)}')
    

In [40]:
import random

In [41]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [43]:
def evaluate(model, val_dl):
    model.eval()
    val_loss = 0
    predictions, true_vals = [], []
    for batch in tqdm(val_dl):
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        with torch.no_grad():
            outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]
        val_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    val_loss_avg = val_loss/len(val_dl)
    prediction = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    return val_loss_avg, predictions, true_vals


In [44]:
!mkdir Models

In [45]:
def fit(model, epochs, train_dl, val_dl, optimizer):
    for epoch in range(epochs):
        model.train()
        training_loss = 0
        tqdm_progress = tqdm(
            train_dl,
            desc=f'Epoch {epoch+1}',
            leave=False,
            disable=False
            )
        for batch in tqdm_progress:
            model.zero_grad()
            batch = tuple(b.to(device) for b in batch)
            inputs = {
                'input_ids' : batch[0],
                'attention_mask' : batch[1],
                'labels' : batch[2]
            }
            outputs = model(**inputs)
            loss = outputs[0]
            training_loss += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            tqdm_progress.set_postfix({'training_loss': f'{loss.item()/len(batch):.3f}'})
        
        torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')
        tqdm.write(f'Epoch {epoch}')
        training_loss_avg = training_loss/len(train_dl)
        tqdm.write(f'Training Loss: {training_loss_avg}')

        val_loss, val_preds, val_true = evaluate(model, val_dl)
        val_preds = np.vstack(np.array(val_preds))
        val_f1 = f1_score_func(val_preds, val_true)
        tqdm.write(f'Val Loss: {val_loss}')
        tqdm.write(f'F1 Score (weighted): {val_f1}')


In [46]:
fit(model, EPOCHS, train_dl, val_dl, optim)

Epoch 0
Training Loss: 1.1084835447371006



Val Loss: 0.780684517962592
F1 Score (weighted): 0.7291634912867575


Epoch 1
Training Loss: 0.7424834750592708



Val Loss: 0.6734176874160767
F1 Score (weighted): 0.7504270486368577


Epoch 2
Training Loss: 0.6232635647058486


...


Val Loss: 0.4129563069769314
F1 Score (weighted): 0.8586023452079213


Epoch 8
Training Loss: 0.22140912283211947



Val Loss: 0.4189474284648895
F1 Score (weighted): 0.8549054595785774


Epoch 9
Training Loss: 0.2065452178940177



Val Loss: 0.4205376569713865
F1 Score (weighted): 0.8561143209156719


In [47]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [48]:
!cp Models/BERT_ft_epoch9.model /gdrive/My\ Drive/

In [49]:
model.load_state_dict(
    torch.load('Models/BERT_ft_epoch9.model')
)

<All keys matched successfully>

In [50]:
_, predictions, true_vals = evaluate(model, val_dl)

In [51]:
print_acc_per_class(np.vstack(predictions), true_vals)

Class: happy
Accuracy: 164/171
Class: not-relevant
Accuracy: 22/32
Class: angry
Accuracy: 6/9
Class: disgust
Accuracy: 0/1
Class: sad
Accuracy: 0/5
Class: surprise
Accuracy: 2/5
